## Red neuronal utilizando Keras

Ahora construiremos una red neuronal utilizando la libreria de Keras para implementar tensorflow.
Utilizaremos la base de datos sin valores faltantes para entrenar el modelo, y mediante validación cruzada probaremos su efectividad.

### Cargando librerias necesarias

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np # linear algebra
import pandas as pd
import keras
import os
import qgrid
import functools


In [2]:
train = pd.read_csv('data/dataset_tae_final_no_na_mod.csv')
test = pd.read_csv('data/test_tae_no_na_mod.csv')

In [59]:
qqview = qgrid.show_grid(train)
qqview

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Definimos nuestra variable objetivo y nuestras caracteristicas

In [3]:
column = list(train.columns)
features = column[:-1]
label = column[-1]
#Chequeo de tipos de las variables
train.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
marital_status    object
ocupation         object
ethnicity         object
gender            object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
dtype: object

Vemos que tenemos variables categoricas, debemos transformarlas a variables dummies par apoder crear la red neuronal:

In [4]:
catego_columns = ['education', 'workclass', 'marital_status', 
             'ethnicity', 'income','gender',
                  'native_country', 'ocupation']
#Transformamos variables object a categoricas
for col in catego_columns:
    train[col] = pd.Categorical(train[col])
    test[col] = pd.Categorical(test[col])

#Transformamos nuestra variable de objetivo para la clasificacion
train[label] = train[label].cat.codes
test[label] = test[label].cat.codes

#Creamos variables dummies con las categoricas
train_dataset=pd.get_dummies(train)
test_dataset=pd.get_dummies(test)

Chequeamos que las variables de nuestro conjunto de entrenamiento sean las mismas que las de validación

In [5]:
for elem in list(train_dataset.columns):
    if not((elem in list(test_dataset.columns))):
        print(elem)
#Everything is fine

In [6]:
columns = train_dataset.columns
columns
#target = train_dataset.pop("income")

Index(['age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week',
       'income', 'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Private', 'workclass_ Self-emp-inc',
       'workclass_ Self-emp-not-inc', 'workclass_ State-gov',
       'workclass_ Without-pay', 'education_ 10th', 'education_ 11th',
       'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th',
       'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm',
       'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate',
       'education_ HS-grad', 'education_ Masters', 'education_ Preschool',
       'education_ Prof-school', 'education_ Some-college',
       'marital_status_ Divorced', 'marital_status_ Never-married',
       'marital_status_ Separated', 'marital_status_ Widowed',
       'marital_status_Married', 'ocupation_ Adm-clerical',
       'ocupation_ Armed-Forces', 'ocupation_ Craft-repair',
       'ocupation_ Exec-managerial', 'ocupation_ Farming-fishi

Finalmente convertimos nuestra base de datos al formato de tensorflow.

In [7]:
df = train_dataset.copy(deep=True)
target = df.pop('income')

In [8]:
final_train = tf.data.Dataset.from_tensor_slices((train_dataset.values, target.values))
final_train = final_train.shuffle(len(train)).batch(64)
type(final_train)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [10]:
train_dataset.to_csv('train_dummies.csv')

### Creando la red neuronal

In [12]:

def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [19]:
x_train = train_dataset.copy(deep=True)
x_val = test_dataset.copy(deep=True)
y_train = x_train.pop('income')
y_val = x_val.pop('income')
temp = x_val.values
temp[0:20,0:10] 
#val_data = tf.data.Dataset.from_tensor_slices((x_test.values, y_test.values))

model = get_compiled_model()

model.fit(x_train, y_train,epochs = 20, 
          steps_per_epoch=20)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=       age  fnlwgt  capital_gain  capital_loss  hours_per_week  \
0       38  215646             0             0              40   
1       53  234721             0             0              40   
2       28  338409             0             0              40   
3       37  284582             0             0              40   
4       49  160187             0             0              16   
5       52  209642             0             0              45   
6       31   45781         14084             0              50   
7       42  159449          5178             0              40   
8       37  280464             0             0              80   
9       30  141297             0             0              40   
10      23  122272             0             0              30   
11      32  205019             0             0              50   
12      34  245487             0          

In [18]:
y_pred=model.predict(final_train)
y_pred =(y_pred>0.5)

CancelledError:  [Op:StatefulPartitionedCall]

## Data cross validation

In [ ]:
model.predict()

In [28]:
print("hola")

hola


In [25]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])
model.fit(final_train, epochs = 20, steps_per_epoch=472)

Epoch 1/20


RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

## Entrenando el modelo

In [20]:
def loss(model, x, y):
  y_ = model(x)
  return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)


l = loss(model, features, target)
print("Loss test: {}".format(l))

NameError: name 'model' is not defined

In [50]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])

['age',
 'workclass',
 'fnlwgt',
 'education',
 'marital_status',
 'ocupation',
 'ethnicity',
 'gender',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country']